# Retrieval Augmented Generation (RAG) with OpenAI and Qdrant

In the ever-evolving landscape of AI, the consistency and reliability of Large Language Models (LLMs) remain a challenge. While these models can understand statistical relationships between words, they often fail to provide accurate factual responses. Because their internal knowledge may not be accurate, outputs can range from spot-on to nonsensical. Retrieval Augmented Generation (RAG) is a framework designed to bolster the accuracy of LLMs by grounding them in external knowledge bases. In this example, we'll demonstrate a streamlined  implementation of the RAG pipeline using only Qdrant and OpenAI SDKs. By harnessing Flag embedding's power, we can bypass additional frameworks' overhead.
    
This example assumes you understand the architecture necessary to carry out RAG. If this is new to you, please look at some introductory readings:
* [Retrieval-Augmented Generation: To add knowledge](https://eugeneyan.com/writing/llm-patterns/#retrieval-augmented-generation-to-add-knowledge)

## Prerequisites

Let's start setting up all the pieces to implement the RAG pipeline. We will only use Qdrant and OpenAI SDKs, without any third-party libraries.

### Preparing the environment

We need just a few dependencies to implement the whole application, so let's start with installing the dependencies.

In [22]:
# To run in Google Colab, need to install these libraries, then restart session
!pip install cohere tiktoken protobuf==3.20.3 typing-extensions==4.5.0 qdrant-client fastembed openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


[Qdrant](https://qdrant.tech) will act as a knowledge base providing the context information for the prompts we'll be sending to the LLM. There are various ways of running Qdrant, but we'll simply use the Docker container.

In [23]:
# Docker
# !docker run -p "6333:6333" -p "6334:6334" --name "rag-openai-qdrant" --rm -d qdrant/qdrant:latest

### Creating the collection

Qdrant [collection](https://qdrant.tech/documentation/concepts/collections/) is the basic unit of organizing your data. Each collection is a named set of points (vectors with a payload) among which you can search. After connecting to our running Qdrant container, we can check whether we already have some collections.

In [24]:
import qdrant_client

# Docker
# client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
# client.get_collections()

# Memory
client = qdrant_client.QdrantClient(":memory:")
client.get_collections()

CollectionsResponse(collections=[])

In [25]:
client.set_model("intfloat/multilingual-e5-large")

In [26]:
client.recreate_collection(
    collection_name="knowledge-base",
    vectors_config=client.get_fastembed_vector_params(),
)

True

### Building the knowledge base

Qdrant will use vector embeddings of our facts to enrich the original prompt with some context. Thus, we need to store the vector embeddings and the texts used to generate them. All our facts will have a JSON payload with a single attribute and look as follows:

```json
{
    "document": "Binary Quantization is a method of reducing the memory usage even up to 40 times!"
}
```

This structure is required by [FastEmbed](https://qdrant.github.io/fastembed/), a library that simplifies managing the vectors, as you don't have to calculate them on your own. It's also possible to use an existing collection, However, all the code snippets will assume this data structure. Adjust your examples to work with a different schema.

FastEmbed will automatically create the collection if it doesn't exist. Knowing that we are set to add our documents to a collection, which we'll call `knowledge-base`.

In [27]:
documents = [
        "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
        "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
        "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
        "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
        "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
        "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
        "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
        "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
]

In [28]:
metadata = [{"text": item} for item in documents]

client.add(
    collection_name="knowledge-base",
    documents=documents,
    metadata=metadata,
)

['6c80c3f32a7147c79d2e123f6b83a5e4',
 'c48ccf2f89204e62ac61568113438d34',
 '0955887bca8a47d5a2146f3a0b1170d4',
 '464c5106c3b5405499512fe61e3747da',
 'b6aedadb4f1a403c8971a2675e7406cf',
 'fd4670a94f5849af86d3cb5824082dd5',
 '082d222c75f04c4cb710a6261b2c0076',
 'd26819716fd24066adc8a205f8d25208']

## Retrieval Augmented Generation

RAG changes the way we interact with Large Language Models. We're converting a knowledge-oriented task, in which the model may create a counterfactual answer, into a language-oriented task. The latter expects the model to extract meaningful information and generate an answer. LLMs, when implemented correctly, are supposed to be carrying out language-oriented tasks.

The task starts with the original prompt sent by the user. The same prompt is then vectorized and used as a search query for the most relevant facts. Those facts are combined with the original prompt to build a longer prompt containing more information.

But let's start simply by asking our question directly.

In [29]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

Using OpenAI API requires providing the API key. Our example demonstrates setting the `OPENAI_API_KEY` using an environmental variable.

In [30]:
import os
# from google.colab import userdata

# Fill the environmental variable with your own Open Router API key
# See: https://openrouter.ai/keys
os.environ["OPENAI_API_KEY"] = ''

Now we can finally call the completion service.

In [31]:
from openai import AsyncOpenAI

openai = AsyncOpenAI()
openai.base_url = "https://openrouter.ai/api/v1"

completion = await openai.chat.completions.create(
    model="mistralai/mistral-7b-instruct", # "gpt-3.5-turbo" # Cost too much
    messages=[
        {"role": "user", "content": prompt},
    ]
)

print(completion.choices)

[Choice(finish_reason=None, index=None, logprobs=None, message=ChatCompletionMessage(content="To build a web service using vector embeddings for search, you would need the following tools:\n\n1. Vector Embedding Model: You will need a pre-trained or trained vector embedding model like Word2Vec or GloVe. This model will be used to convert text into numerical vectors that can be used for search.\n2. Web Service Framework: You will need a web service framework like Flask or Django to create the web service. This framework will help you create the API endpoints and handle requests and responses.\n3. Search Engine: You will need a search engine like Elasticsearch or Solr to index and search the documents. This search engine will be used to search the indexed documents based on the user's query.\n4. Data Storage: You will need a data storage system like MongoDB or PostgreSQL to store the documents. This system will be used to store the indexed documents and retrieve them when needed.\n5. Dep

In [32]:
print(completion.choices[0].message.content)

To build a web service using vector embeddings for search, you would need the following tools:

1. Vector Embedding Model: You will need a pre-trained or trained vector embedding model like Word2Vec or GloVe. This model will be used to convert text into numerical vectors that can be used for search.
2. Web Service Framework: You will need a web service framework like Flask or Django to create the web service. This framework will help you create the API endpoints and handle requests and responses.
3. Search Engine: You will need a search engine like Elasticsearch or Solr to index and search the documents. This search engine will be used to search the indexed documents based on the user's query.
4. Data Storage: You will need a data storage system like MongoDB or PostgreSQL to store the documents. This system will be used to store the indexed documents and retrieve them when needed.
5. Deployment Platform: You will need a deployment platform like Heroku or AWS to deploy the web service. 

### Extending the prompt

Even though the original answer sounds credible, it didn't answer our question correctly. Instead, it gave us a generic description of an application stack. To improve the results, enriching the original prompt with the descriptions of the tools available seems like one of the possibilities. Let's use a semantic knowledge base to augment the prompt with the descriptions of different technologies!

In [33]:
results = client.query(
    collection_name="knowledge-base",
    query_text=prompt,
    limit=3,
)
results

[QueryResponse(id='082d222c75f04c4cb710a6261b2c0076', embedding=None, metadata={'document': 'SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.', 'text': 'SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.'}, document='SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. 

In [34]:
[result.metadata['document'] for result in results]

['SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.',
 'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!',
 'NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.']

We used the original prompt to perform a semantic search over the set of tool descriptions. Now we can use these descriptions to augment the prompt and create more context.

In [35]:
context = "\n".join(r.document for r in results)
print(context)

SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.
Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.


Finally, let's build a metaprompt, the combination of the assumed role of the LLM, the original question, and the results from our semantic search that will force our LLM to use the provided context.

By doing this, we effectively convert the knowledge-oriented task into a language task and hopefully reduce the chances of hallucinations. It also should make the response sound more relevant.

In [36]:
metaprompt = f"""
You are a software architect.
Answer the following question using the provided context.
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: {prompt.strip()}

Context:
{context.strip()}

Answer:
"""

# Look at the full metaprompt
print(metaprompt)


You are a software architect.
Answer the following question using the provided context.
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: What tools should I need to use to build a web service using vector embeddings for search?

Context:
SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.
Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
NGINX is a free, open-

Our current prompt is much longer, and we also used a couple of strategies to make the responses even better:

1. The LLM has the role of software architect.
2. We provide more context to answer the question.
3. If the context contains no meaningful information, the model shouldn't make up an answer.

Let's find out if that works as expected.

In [37]:
completion = await openai.chat.completions.create(
    model="mistralai/mistral-7b-instruct",
    messages=[
        {"role": "user", "content": metaprompt},
    ],
    timeout=10.0,
)
completion.choices[0].message.content

'To build a web service using vector embeddings for search, you will need to use the following tools:\n\n1. SentenceTransformers: This is a Python framework that provides pre-trained sentence and text embeddings for more than 100 languages. You can use this framework to compute embeddings for your text data.\n2. Qdrant: This is a vector database and similarity search engine that can be used to index and search your embeddings. Qdrant provides a simple API for searching the nearest high-dimensional vectors to a given query.\n3. NGINX: This is an open-source HTTP server and reverse proxy that can be used to host your web service. NGINX is known for its high performance, stability, and simple configuration.\n\nOnce you have these tools set up, you can use them together to build a web service that uses vector embeddings for search. You can use SentenceTransformers to compute embeddings for your text data, and then use Qdrant to index and search the embeddings. Finally, you can use NGINX to

### Testing out the RAG pipeline

By leveraging the semantic context we provided our model is doing a better job answering the question. Let's enclose the RAG as a function, so we can call it more easily for different prompts.

In [38]:
async def rag(question: str, n_points: int = 3) -> str:
    results = client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect.
    Answer the following question using the provided context.
    If you can't find the answer, do not pretend you know it, but answer "I don't know".

    Question: {question.strip()}

    Context:
    {context.strip()}

    Answer:
    """

    completion = await openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion.choices[0].message.content

Now it's easier to ask a broad range of questions.

In [39]:
await rag("What can the stack for a web api look like?")

'The stack for a web API using FastAPI, NGINX, and Qdrant can look like the following:\n\n1. NGINX as a reverse proxy: NGINX can be used as a reverse proxy server to handle incoming requests and distribute them to multiple instances of the FastAPI application running on different machines or containers. NGINX can also handle load balancing, caching, and SSL termination.\n\n2. FastAPI: FastAPI is the web framework for building the API itself. It provides an efficient, high-performance way to handle HTTP requests and responses based on standard Python type hints. FastAPI supports async operations, automatic validation of request data, automatic generation of documentation, and more.\n\n3. Qdrant: Qdrant can be used as a vector database and vector similarity search engine to power certain search functionalities of the web API. It can be integrated into the FastAPI application as a separate service or module. Qdrant allows for searching high-dimensional vectors and can be useful for applic

In [40]:
await rag("Where is the nearest grocery store?")

"I don't know."

Our model can now:

1. Take advantage of the knowledge in our vector datastore.
2. Answer, based on the provided context, that it can not provide an answer.

We have just shown a useful mechanism to mitigate the risks of hallucinations in Large Language Models.

### Cleaning up the environment

If you wish to continue playing with the RAG application we created, don't do the code below. However, it's always good to clean up the environment, so nothing is left dangling. We'll show you how to remove the Qdrant container.

In [41]:
# !docker kill rag-openai-qdrant
# !docker rm rag-openai-qdrant